<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Position-space" data-toc-modified-id="Position-space-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Position space</a></span><ul class="toc-item"><li><span><a href="#Translation-of-COM-instead-of-rotation-around-origin" data-toc-modified-id="Translation-of-COM-instead-of-rotation-around-origin-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Translation of COM instead of rotation around origin</a></span></li><li><span><a href="#Determine-position-index" data-toc-modified-id="Determine-position-index-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Determine position index</a></span></li></ul></li><li><span><a href="#Orientation-space" data-toc-modified-id="Orientation-space-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Orientation space</a></span><ul class="toc-item"><li><span><a href="#Determine-internal-coordinate-system-(principal-axes)" data-toc-modified-id="Determine-internal-coordinate-system-(principal-axes)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Determine internal coordinate system (principal axes)</a></span></li><li><span><a href="#Realize-that-determining-principal-axes-isn't-sufficient" data-toc-modified-id="Realize-that-determining-principal-axes-isn't-sufficient-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Realize that determining principal axes isn't sufficient</a></span></li><li><span><a href="#Quaternion-index" data-toc-modified-id="Quaternion-index-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Quaternion index</a></span></li></ul></li><li><span><a href="#Full-conversion-3N-coordinates-<->-7D-gridpoint" data-toc-modified-id="Full-conversion-3N-coordinates-<->-7D-gridpoint-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Full conversion 3N coordinates &lt;-&gt; 7D gridpoint</a></span></li><li><span><a href="#MSM" data-toc-modified-id="MSM-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>MSM</a></span></li></ul></div>

In [1]:
import sys
import os

try:
    os.chdir(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup/molgri")
    sys.path.append(r"/home/hanaz63/PAPER_MOLECULAR_ROTATIONS_2022/nobackup")
except FileNotFoundError:
    os.chdir(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    sys.path.append(r"D:\HANA\phD\PAPER_2022\molecularRotationalGrids")
    
import warnings
warnings.filterwarnings("ignore")

In [14]:
import numpy as np
from numpy.typing import NDArray
import MDAnalysis as mda
from scipy.spatial.distance import cdist
from scipy.spatial.transform import Rotation
from MDAnalysis.analysis.base import AnalysisFromFunction
import seaborn as sns

from molgri.plotting.widgets import ViewManager
from molgri.paths import PATH_OUTPUT_PT, PATH_INPUT_BASEGRO
from molgri.space.fullgrid import FullGrid
from molgri.space.utils import normalise_vectors, k_argmax_in_array, distance_between_quaternions, q_in_upper_sphere

sns.set_context("notebook")

## Position space



### Translation of COM instead of rotation around origin

In [15]:
# generated a file with
# python -m molgri.scripts.generate_pt -m1 H2O -m2 H2O -o "12" -b "8" -t "linspace(0.2, 1, 5)" --recal
pt_name = "H2O_H2O_0528"
pt_universe = mda.Universe(f"{PATH_OUTPUT_PT}{pt_name}.gro",
                           f"{PATH_OUTPUT_PT}{pt_name}.trr")
len_pt = len(pt_universe.trajectory)

vm = ViewManager(pt_universe)
# plot every 8th frame
# because we have 8 orientations, every 8th structure should be translated version of each other
every_eighth_0 = np.arange(0, len_pt, 8)
print(list(every_eighth_0))
vm.plot_frames_overlapping(every_eighth_0[20:30])

In [16]:
vm.fresh_view()
every_eighth_3 = np.arange(3, len_pt, 8)
# similar thing for another class of identical orientations
print(list(every_eighth_3))
vm.plot_frames_overlapping(every_eighth_3[20:30])

### Determine position index

In [17]:
# separately determine index in the radial direction and on the sphere, then combine

def assign_2_t_grid(my_trajectory: mda.Universe, t_grid_points: NDArray,
                         second_molecule: str) -> NDArray:
    """
    Given a trajectoryand an array of available radial (t-grid) points, assign each frame of the trajectory
    to the closest radial point.
    
    Args:
        my_trajectory: a Universe of two molecules
        radial_grid_points: a sorted array of available radial grid distances in A like [2, 4, 6]
        second_molecule: a string that tdefines which part of the Universe is a moving molecule
    
    Returns:
        an integer array as long as the trajectory, each element an index of the closest point of the radial grid
        like [0, 0, 0, 1, 1, 1, 2 ...] (for a PT with 3 orientations)
    """
    t_selection = AnalysisFromFunction(lambda ag: np.argmin(np.abs(t_grid_points-np.linalg.norm(ag.center_of_mass())), axis=0),
                                     my_trajectory.trajectory,
                                     my_trajectory.select_atoms(second_molecule))
    t_selection.run()
    t_indices = t_selection.results['timeseries'].flatten()
    return t_indices

def assign_2_o_grid(my_trajectory: mda.Universe, o_grid_points: NDArray, 
                           second_molecule: str) -> NDArray:
    """
    Assign every frame of the trajectory (or PT) to the best fitting point of position grid

    Returns:
        an array of position grid indices
    """
    # now using a normalized com and a metric on a sphere, determine which of o_grid_points is closest
    o_selection = AnalysisFromFunction(lambda ag: np.argmin(cdist(o_grid_points, normalise_vectors(
        ag.center_of_mass())[np.newaxis, :], metric="cos"), axis=0),
                                     my_trajectory.trajectory,
                                     my_trajectory.select_atoms(second_molecule))
    o_selection.run()
    o_indices = o_selection.results['timeseries'].flatten()
    return o_indices

def assign_2_position_grid(my_trajectory: mda.Universe, o_grid_points: NDArray, t_grid_points: NDArray,
                         second_molecule: str):
    """
    Combine assigning to t_grid and o_grid.
    """
    t_assignments = assign_2_t_grid(my_trajectory=my_trajectory, t_grid_points=t_grid_points, 
                                    second_molecule=second_molecule)
    o_assignments = assign_2_o_grid(my_trajectory=my_trajectory, o_grid_points=o_grid_points,
                                   second_molecule=second_molecule)
    # sum up the layer index and o index correctly
    return np.array(t_assignments * len(o_grid_points) + o_assignments, dtype=int)


In [18]:
# test with a PT

my_second_molecule = "bynum 4:6"
my_full_grid = FullGrid(b_grid_name="8", o_grid_name="12", t_grid_name="linspace(0.2, 1, 5)")
position_assignments = assign_2_position_grid(pt_universe, my_full_grid.get_o_grid().get_grid_as_array(), 
                       my_full_grid.get_radii(), my_second_molecule)
print(position_assignments)
# test: a PT should have the first 8 elements in position 0, then 8 in position 1 ...
expected_assignments = np.repeat(np.arange(12*5), 8)
assert np.all(position_assignments == expected_assignments)

In [19]:
# test with a real trajectory: see how long it takes
gro_name = "H2O_H2O_0095"
traj_name = "H2O_H2O_0095_25000"
traj_universe = mda.Universe(f"{PATH_OUTPUT_PT}{gro_name}.gro", f"{PATH_OUTPUT_PT}{traj_name}.xtc")

print(f"Assigning 25000 frames to 25x20={25*20} positions")
traj_full_grid = FullGrid(b_grid_name="20", o_grid_name="25", t_grid_name="linspace(0.2, 1.5, 20)")
position_assignments = assign_2_position_grid(traj_universe, traj_full_grid.get_o_grid().get_grid_as_array(), 
                       traj_full_grid.get_radii(), my_second_molecule)

In [20]:
# make a histogram of most populated positions
sns.histplot(position_assignments, stat="count", bins=np.arange(501))

popular, counts = np.unique(position_assignments, return_counts=True)
max_counts = k_argmax_in_array(counts, 10)
print(f"Most popular assignments are {popular[max_counts]} with population {counts[max_counts]}")

In [21]:
# plot some frames of a real trajectory that belong to the same position grid point
vm = ViewManager(traj_universe)
# plot a selection (because too many) that are all assigned to grid point 26
assigned_to_26 = np.where(position_assignments == 26)[0]
# show in VMD for a better view
print(list(assigned_to_26))
vm.plot_frames_overlapping(np.random.choice(assigned_to_26, 30))

In [22]:
# try another one
vm.fresh_view()
# plot a selection (because too many) that are all assigned to grid point 477
assigned_to_477 = np.where(position_assignments == 477)[0]
# show in VMD for a better view
print(list(assigned_to_477))
vm.plot_frames_overlapping(np.random.choice(assigned_to_477, 30))

## Orientation space

### Determine internal coordinate system (principal axes)

In [23]:
# determine principal axes
second_molecule = "bynum 4:6"

# principal axes of the first frame
pa_zero = pt_universe.select_atoms(second_molecule).principal_axes()
com_zero = pt_universe.select_atoms(second_molecule).center_of_mass()
print(np.round(pa_zero, 3))
print(com_zero+pa_zero[0])

# display principal axes of first frame
vm = ViewManager(pt_universe)
vm.fresh_view()
vm.plot_ith_frame(0)
vm.add_principal_axes(com_zero, pa_zero)
vm.view



In [24]:
vm.fresh_view()

# principal axes change in relation to the stationary coordinate system when the molecule is re-oriented
# but they remain the same in relation to the molecule itself

# using some selected PT frames for demonstration
for i in [20, 63, 284]:
    pt_universe.trajectory[i]
    pa_current = pt_universe.select_atoms(second_molecule).principal_axes()
    com_current = pt_universe.select_atoms(second_molecule).center_of_mass()
    print(np.round(pa_current, 3))

    # display this
    vm.plot_ith_frame(i)
    vm.add_principal_axes(com_current, pa_current)
vm.view

... well, almost the same

### Realize that determining principal axes isn't sufficient

In [25]:
# LESSION LEARNED: principal axes are axes and NOT vectors - their direction is NOT clearly defined
# see also: https://math.stackexchange.com/questions/145023/rotation-matrix-from-an-inertia-tensor

# python -m molgri.scripts.generate_pt -m1 H2O -m2 H2O -o "1" -b "20" -t "1" --recal
from molgri.space.rotobj import SphereGrid4DFactory
b_selection = SphereGrid4DFactory().create(alg_name="cube4D", N=20).get_grid_as_array()
b_universe = mda.Universe(f"{PATH_OUTPUT_PT}H2O_H2O_0407.gro", f"{PATH_OUTPUT_PT}H2O_H2O_0407.xtc")

vm = ViewManager(b_universe)
vm.fresh_view()
reference_structure = mda.Universe(f"{PATH_INPUT_BASEGRO}H2O.gro")
reference_principal_axes = reference_structure.atoms.principal_axes()

for i in range(17, 18):
    b_universe.trajectory[i]
    reference_structure = mda.Universe(f"{PATH_INPUT_BASEGRO}H2O.gro")
    reference_principal_axes = reference_structure.atoms.principal_axes()
    my_R = Rotation(b_selection[i]).as_matrix()
    expected_pa = Rotation(b_selection[i]).apply(reference_principal_axes)
    #expected_pa = (my_R @ reference_principal_axes.T).T

    print("my R @ ref", np.round(expected_pa, 3))

    current_principal_axes = b_universe.select_atoms(second_molecule).principal_axes()
    com_current = b_universe.select_atoms(second_molecule).center_of_mass()
    print("end_pa", np.round(current_principal_axes, 3))
    vm.plot_ith_frame(i)
    vm.add_principal_axes(com_current, expected_pa)
    vm.add_principal_axes(com_current, current_principal_axes)
vm.view

In [26]:
# LESSONS LEARNED
# 1. symmetry of water is a problem
# 2. need to save in a higher precision file trr
# 3. need to round structures read from files otherwise values very close to zero can throw off 
#    quaternion projection to correct hemisphere

def determine_positive_directions(current_universe, second_molecule):
    pas = current_universe.select_atoms(second_molecule).principal_axes()
    com = current_universe.select_atoms(second_molecule).center_of_mass()
    directions = [0, 0, 0]
    for atom_pos in current_universe.select_atoms(second_molecule).positions:
        for i, pa in enumerate(pas):
            # need to round to avoid problems - assigning direction with atoms very close to 0
            cosalpha = np.round(pa.dot(atom_pos-com), 6)
            directions[i] = np.sign(cosalpha)
        if not np.any(np.isclose(directions,0)):
            break
    # TODO: if exactly one unknown use the other two and properties of righthanded systems to get third
    if np.sum(np.isclose(directions,0)) == 1:
        # only these combinations of directions are possible in righthanded coordinate systems
        allowed_righthanded = [[1, 1, 1], [-1, 1, -1], [1, -1, -1], [-1, -1, 1]]
        for ar in allowed_righthanded:
            # exactly two identical (and the third is zero)
            if np.sum(np.isclose(ar, directions)) == 2:
                directions = ar
                break
    # if two (or three - that would just be an atom) unknowns raise an error
    elif np.sum(np.isclose(directions,0)) > 1:
        raise ValueError("All atoms perpendicular to at least one of principal axes, can´t determine direction.")
    return np.array(directions)

# saved PT
my_new_u = mda.Universe(f"{PATH_OUTPUT_PT}glucose_glucose_0025.gro", f"{PATH_OUTPUT_PT}glucose_glucose_0025.trr")
b_selection = SphereGrid4DFactory().create(alg_name="cube4D", N=20).get_grid_as_array()
second_molecule_glu = "bynum 13:24"

# reference structure
reference_structure = mda.Universe(f"{PATH_INPUT_BASEGRO}glucose.xyz")
second_molecule_ref = "bynum 1:12"
reference_principal_axes = reference_structure.atoms.principal_axes().T
inverse_pa = np.linalg.inv(reference_principal_axes)
reference_direction = determine_positive_directions(reference_structure, second_molecule_ref)

# frame 0: q3 in direction C-O, frame 1: q3 in direction O
for i in range(20):
    my_new_u.trajectory[i]
    com_current = my_new_u.select_atoms(second_molecule_glu).center_of_mass()
    current_principal_axes = my_new_u.select_atoms(second_molecule_glu).principal_axes().T
    current_directions = determine_positive_directions(my_new_u, second_molecule_glu)
    direction = current_directions * reference_direction
    produkt = np.multiply(current_principal_axes, np.tile(direction, (3, 1))) @ inverse_pa
    calc_quat = np.round(Rotation.from_matrix(produkt).as_quat(), 6) 
    if not q_in_upper_sphere(calc_quat):
        calc_quat = -calc_quat
    print(current_directions)
    print(np.allclose(b_selection[i], calc_quat, atol=1e-5, rtol=1e-5), b_selection[i], calc_quat)

In [27]:
#python -m molgri.scripts.generate_pt -m1 H2O_a -m2 H2O_a -o "1" -b "20" -t "0.3" --recal --extension_structure gro --extension_trajectory trr


# saved PT
my_new_u = mda.Universe(f"{PATH_OUTPUT_PT}H2O_a_H2O_a_0000.gro", f"{PATH_OUTPUT_PT}H2O_a_H2O_a_0000.trr")
b_selection = SphereGrid4DFactory().create(alg_name="cube4D", N=20).get_grid_as_array()
second_molecule_glu = "bynum 4:6"

# reference structure
reference_structure = mda.Universe(f"{PATH_INPUT_BASEGRO}H2O_a.gro")
second_molecule_ref = "bynum 1:3"
reference_principal_axes = reference_structure.atoms.principal_axes().T
inverse_pa = np.linalg.inv(reference_principal_axes)
reference_direction = determine_positive_directions(reference_structure, second_molecule_ref)

# frame 0: q3 in direction C-O, frame 1: q3 in direction O
for i in range(20):
    my_new_u.trajectory[i]
    com_current = my_new_u.select_atoms(second_molecule_glu).center_of_mass()
    current_principal_axes = my_new_u.select_atoms(second_molecule_glu).principal_axes().T
    current_directions = determine_positive_directions(my_new_u, second_molecule_glu)
    direction = current_directions * reference_direction
    produkt = np.multiply(current_principal_axes, np.tile(direction, (3, 1))) @ inverse_pa
    calc_quat = np.round(Rotation.from_matrix(produkt).as_quat(), 6) 
    if not q_in_upper_sphere(calc_quat):
        calc_quat = -calc_quat
    print(np.allclose(b_selection[i], calc_quat, atol=1e-6, rtol=1e-5), b_selection[i], calc_quat)

In summary:
- save and use (pseudo)trajectories in high-precision TRR format
- determine principal axes of the reference structure and every frame of the PT
- determine direction of each PA from the coordinates of the first atom that isn't perpendicular to it
- if a molecule is planar, you can still determine directions from right-handedness of PAs

TODO:
- now assign to gridpoints
- need a better symmetry handling for linear molecules

### Quaternion index

In [28]:
def assign_2_b_grid(my_trajectory: mda.Universe, b_grid_points: NDArray, reference_universe: mda.Universe,
                           second_molecule: str) -> NDArray:
    """
    Assign every frame of the trajectory to the closest quaternion from the b_grid_points.
    
    Args:
        my_trajectory: consists of two molecules and multiple frames
        b_grid_points: a (N, 4) array of unit quaternions
        reference_universe: consists of only one molecule in a single frame
        second_molecule: defines which molecule interests us in my_trajectory
    """
    # find PA and direction of reference structure
    reference_principal_axes = reference_universe.atoms.principal_axes().T
    inverse_pa = np.linalg.inv(reference_principal_axes)
    reference_direction = determine_positive_directions(reference_universe, "all")
    
    # find PA and direction along trajectory
    pa_frames =  AnalysisFromFunction(lambda ag: ag.principal_axes().T, my_trajectory.trajectory,
                                     my_trajectory.select_atoms(second_molecule))
    pa_frames.run()
    pa_frames = pa_frames.results['timeseries']
    
    direction_frames = AnalysisFromFunction(lambda ag: np.tile(determine_positive_directions(ag, "all"), (3, 1)), 
                                            my_trajectory.trajectory,
                                            my_trajectory.select_atoms(second_molecule))
    direction_frames.run()
    direction_frames = direction_frames.results['timeseries']
    directed_pas = np.multiply(pa_frames, direction_frames)
    produkt = np.matmul(directed_pas, inverse_pa)
    # get the quaternions that caused the rotation from reference to each frame
    calc_quat = np.round(Rotation.from_matrix(produkt).as_quat(), 6)
    # now using a quaternion metric, determine which of b_grid_points is closest

    b_indices = np.argmin(cdist(b_grid_points, calc_quat, metric=distance_between_quaternions), axis=0)
    # almost everything correct but the order is somehow mixed???
    return b_indices

# water PT example
H2O_pt_name = "H2O_H2O_0528"
H2O_pt_universe = mda.Universe(f"{PATH_OUTPUT_PT}{H2O_pt_name}.gro",
                           f"{PATH_OUTPUT_PT}{H2O_pt_name}.trr")
H2O_second_mol = "bynum 4:6"
H2O_reference_universe = mda.Universe(f"{PATH_INPUT_BASEGRO}H2O.gro")
H2O_full_grid = FullGrid(b_grid_name="8", o_grid_name="12", t_grid_name="linspace(0.2, 1, 5)", use_saved=False)
H2O_b_grid = my_full_grid.b_rotations.get_grid_as_array()
                                      
assignments_quaternion_pt = assign_2_b_grid(H2O_pt_universe, H2O_b_grid, H2O_reference_universe, H2O_second_mol)
print(assignments_quaternion_pt)
vm = ViewManager(H2O_pt_universe)
# plot a selection (because too many) that are all assigned to grid point 21
assigned_to_26 = np.where(assignments_quaternion_pt == 1)[0]
# show in VMD for a better view
print(list(assigned_to_26))
vm.plot_frames_overlapping(np.random.choice(assigned_to_26, 30))

In [29]:
# test with a real trajectory: see how long it takes
gro_name = "H2O_H2O_0095"
traj_name = "H2O_H2O_0095_25000"
b_selection = SphereGrid4DFactory().create(alg_name="cube4D", N=80).get_grid_as_array()
traj_universe = mda.Universe(f"{PATH_OUTPUT_PT}H2O_H2O_0095.gro", f"{PATH_OUTPUT_PT}H2O_H2O_0095_25000.xtc")

print(f"Assigning 25000 frames to 80 quaternions")
print(b_selection)

quaternion_assignments = assign_2_b_grid(traj_universe, b_selection, H2O_reference_universe, H2O_second_mol)

In [30]:
# make a histogram of most populated quaternions
sns.histplot(quaternion_assignments, stat="count", bins=np.arange(81))

popular, counts = np.unique(quaternion_assignments, return_counts=True)
max_counts = k_argmax_in_array(counts, 3)
print(f"Most popular assignments are {popular[max_counts]} with population {counts[max_counts]}")

In [31]:
# plot some frames of a real trajectory that belong to the same quaternion grid point
vm = ViewManager(traj_universe)
# plot a selection (because too many) that are all assigned to grid point 21
assigned_to_26 = np.where(quaternion_assignments == 15)[0]
# show in VMD for a better view
print(list(assigned_to_26))
vm.plot_frames_overlapping(np.random.choice(assigned_to_26, 30))

## Full conversion 3N coordinates <-> 7D gridpoint

In [ ]:
def full_assignment(position_assignment, quaternion_assignment, num_quaternions):
    return position_assignment*num_quaternions + quaternion_assignment

fg = FullGrid(b_grid_name="40", o_grid_name="42", t_grid_name="linspace(0.2, 1, 20)")
b_grid = fg.b_rotations.get_grid_as_array()
o_grid = fg.get_o_grid().get_grid_as_array()
t_grid = fg.get_radii()

position_assignments = assign_2_position_grid(traj_universe, o_grid, 
                       t_grid, H2O_second_mol)
quaternion_assignments = assign_2_b_grid(traj_universe, b_grid, H2O_reference_universe, H2O_second_mol)

full_assignments = full_assignment(position_assignments, quaternion_assignments, len(b_grid))

In [ ]:
# make a histogram of most populated full grid cells
sns.histplot(full_assignments, stat="count", bins=np.arange(len(fg)+1))

popular, counts = np.unique(full_assignments, return_counts=True)
max_counts = k_argmax_in_array(counts, 10)
print(f"Most popular assignments are {popular[max_counts]} with population {counts[max_counts]}")

In [ ]:
vm = ViewManager(traj_universe)
# plot a selection (because too many) that are all assigned to same grid point
assigned_to_cell = np.where(full_assignments == 3550)[0]
# show in VMD for a better view
print(list(assigned_to_cell))
vm.plot_frames_overlapping(np.random.choice(assigned_to_cell, 30))

In [ ]:
vm = ViewManager(traj_universe)
# plot a selection (because too many) that are all assigned to same grid point
assigned_to_cell = np.where(full_assignments == 4811)[0]
# show in VMD for a better view
print(list(assigned_to_cell))
vm.plot_frames_overlapping(np.random.choice(assigned_to_cell, 30))



## MSM

In [ ]:
from molgri.molecules.transitions import MSM, SimulationHistogram

water_sh = SimulationHistogram(trajectory_name="H2O_H2O_0095_25000", is_pt=False, 
                               second_molecule_selection=H2O_second_mol, 
                               full_grid = fg, use_saved=False)

water_MSM = MSM(water_sh)
water_MSM.assignments = full_assignments
water_MSM.get_transitions_matrix()

In [83]:
# -> direction: INPUT start coordinates (3N), 7D gridpoint; OUTPUT end coordinates (3N)
# <- direction: INPUT end coordinates (3N), start coordinates (3N); OUTPUT 7D gridpoint